In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os

tf.__version__

'2.5.0'

In [2]:
# import tarfile
# tar_train_val = tarfile.open('./data/VOCtrainval_06-Nov-2007.tar')
# tar_train_val.extractall('./data/')

In [3]:
def load_classes_from_txt(file_path):
    
    file_name = os.path.basename(file_path)
    
    class_name = file_name.split('_')[0]
    
    df = pd.read_csv(file_path, sep='\s+| |  ', engine='python', 
                     header=None)
    
    df.columns = ['file_name', class_name]
    
    df.loc[df[class_name]==-1, class_name] = 0
    
    return df
    

In [10]:
# data = load_classes_from_txt('./data/VOCdevkit/VOC2007/ImageSets/Main/aeroplane_train.txt')
# data.head()

In [11]:
# file_name = os.path.basename('./data/VOCdevkit/VOC2007/ImageSets/Main/aeroplane_train.txt')
# file_name.split('_')[0]

In [81]:
folder_path = './data/VOCdevkit/VOC2007/ImageSets/Main/'

def get_dataframe_with_classes_in_image(folder_path):

    for root, _, files in os.walk(folder_path):
        pass
    
    # create dataframes with file name column 
    df_train = pd.read_csv(root+'train.txt', header=None, dtype=str)
    df_train.columns = ['file_name']
    
    df_val = pd.read_csv(root+'trainval.txt', header=None, dtype=str)
    df_val.columns = ['file_name']
    
    df_test = pd.read_csv(root+'val.txt', header=None, dtype=str)
    df_test.columns = ['file_name']
    
    # loops through all file names in the folder
    for file_name in files:
        
        # exclude txt files without information about class apperance in a image
        if file_name not in ['train.txt', 'trainval.txt', 'val.txt']:
            
            #  get information which of dataset's splits the file relates to
            ds_part = file_name.split('_')[1]
            
            file_path = root + file_name
            
            # function for getting dataframe from txt file
            df = load_classes_from_txt(file_path=file_path)
            
            # concatenation
            if ds_part=='train.txt':
                
                df_train = pd.concat([df_train, df.iloc[:, 1]], axis=1)
            
            elif ds_part=='trainval.txt':
                
                df_val = pd.concat([df_val, df.iloc[:, 1]], axis=1)
            
            elif ds_part=='val.txt':
                
                df_test = pd.concat([df_test, df.iloc[:, 1]], axis=1)

    # get column names exluding 'file_name'
    cols = df_train.columns.tolist()[1:]
        
    cols.sort()
    
    cols.insert(0, 'file_name')
    
#     print(cols)

    # re-arange columns order
    df_train = df_train[cols]
    df_val = df_val[cols]
    df_test = df_test[cols]
           
    return df_train, df_val, df_test

df_train, df_val, df_test = get_dataframe_with_classes_in_image(folder_path)

In [82]:
df_train.head()

,file_name,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,...,diningtable,dog,horse,motorbike,person,pottedplant,sheep,sofa,train,tvmonitor
0,000012,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000017,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,000023,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,000026,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000032,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [83]:
df_val.head()

,file_name,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,...,diningtable,dog,horse,motorbike,person,pottedplant,sheep,sofa,train,tvmonitor
0,000005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,000007,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000009,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,000012,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000016,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
df_test.head()

,file_name,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,...,diningtable,dog,horse,motorbike,person,pottedplant,sheep,sofa,train,tvmonitor
0,000005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,000007,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000009,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,000016,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000019,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Apparently authors decided validation images will also include the same images as test part.

2510